Notes from JJ:

* btw you are querying an asteroid, not a KBO, which explains the weird movement! - cool!
* Keep the aladin widget but dont query that way - TODO: Get clarification on how then
* Get the results from SSOIS but then query CADC using the image and RA/DEC
* Use the cadc astroquery `get_url` function then do cutout from RA/DEC
* Have a plot that user can choose the date range
* Display KBOs along same axes, linking the views so when one moves, another moves, e.g. 

date 1 | date 2 | date 3
--- | --- | ---
KBO | KBO | KBO

* Have a circle around KBO - like quasar
* Allow user to click KBO, create `photoutils` aperture with pixels from 


# Interactive KBO Notebook

In [1]:
%matplotlib widget

In [2]:
# ! conda activate tutorials
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py --sys-prefix ipympl
# !jupyter nbextension enable --py --sys-prefix ipyaladin

In [3]:
import math
import numpy as np
import pandas as pd

from ipyaladin import Aladin
from ipywidgets import Layout, Output, Box, widgets

from astropy.table import Table
from astropy.time import Time, TimeDelta
import astropy.units as u
from astroquery.cadc import Cadc


## SSOIS Query

In [105]:
# SSOIS query
# Search by object name
# Date Range Entered: 2006-09-01 to 2006-09-30
objects = ['Yasutani', '2006+RJ43', '45302', '6641+P-L']
# url = ('https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/cadcbin/ssos/ssosclf.pl?lang=en&object={}'
#        '&search=bynameCADC&epoch1=2006+09+01&epoch2=2006+09+30&eellipse=&eunits=arcseconds'
#        '&extres=yes&xyres=yes;format=tsv').format(objects[3])
kbo_name = objects[3]
url = ('https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/cadcbin/ssos/ssosclf.pl?lang=en&object={}'
       '&search=bynameCADC&epoch1=2006+09+01&epoch2=2006+09+30&eellipse=&eunits=arcseconds'
       '&extres=no&xyres=no;format=tsv').format(kbo_name)
data_table = pd.read_csv(url, sep='\t')

# filter table with negative extension values
# data_table = data_table[data_table['Ext'] > 0]

data_table.head()
table = Table.from_pandas(data_table)

table

Image,MJD,Filter,Exptime,Object_RA,Object_Dec,Image_target,Telescope/Instrument,MetaData,Datalink
str7,float64,str8,int64,float64,float64,str6,str12,str73,float64
869870p,53992.3966645426,G.MP9401,30,16.8555741361326,6.32166055369719,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869870p.fits.fz,--
869882p,53992.408535630995,G.MP9401,30,16.8532608567053,6.32249891634057,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869882p.fits.fz,--
869894p,53992.419775212,G.MP9401,30,16.8510706370213,6.3232926805275795,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869894p.fits.fz,--
869906p,53992.431291730994,G.MP9401,30,16.8488264513521,6.3241060026916305,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869906p.fits.fz,--
869918p,53992.4427910773,G.MP9401,30,16.8465856120699,6.32491811208139,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869918p.fits.fz,--
869930p,53992.4546321005,G.MP9401,30,16.8442781913499,6.32575435145283,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869930p.fits.fz,--
869942p,53992.4659541947,G.MP9401,30,16.8420718925934,6.326553942905459,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869942p.fits.fz,--
869954p,53992.4779011889,G.MP9401,30,16.839743821657,6.3273976661847495,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869954p.fits.fz,--
869966p,53992.489347029405,G.MP9401,30,16.837513408871402,6.3282059968760995,Main12,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869966p.fits.fz,--


In [5]:
aladin = Aladin(layout=Layout(width='100%', height='400px'), fov=20)
aladin

Aladin(fov=20.0, layout=Layout(height='400px', width='100%'), options=['allow_full_zoomout', 'coo_frame', 'fov…

In [6]:
aladin.add_table(table)

## CADC Query

In [7]:
# ra_list, dec_list, mjd_list = table_selection['Object_RA'], table_selection['Object_Dec'], table_selection['MJD']
# ext_list = table_selection['Ext']
# TODO: Adjust to search more than just productID (only valid for megaprime i think?)
productId_list = "', '".join(table['Image'])

query = '''SELECT *
FROM caom2.Plane AS Plane 
JOIN caom2.Observation AS Observation ON Plane.obsID = Observation.obsID 
WHERE ( Plane.productID IN ('{}') )
ORDER BY time_bounds_lower ASC'''.format(productId_list)
cadc = Cadc()
job = cadc.run_query(query, 'sync')
results = job.get_results()

cols = [
 'calibrationLevel',
 'position_bounds',
 'position_bounds_size',
 'position_sampleSize',
 'position_dimension_naxis1',
 'position_dimension_naxis2',
 'obsID',
 'planeID',
 'productID',
 'dataProductType',
 'quality_flag',
 'position_bounds_points',
 'proposal_keywords',
 'collection',
 'type',
 'target_name',
 'telescope_name',
 'time_bounds_lower']

results[cols]

calibrationLevel,position_bounds,position_bounds_size,position_sampleSize,position_dimension_naxis1,position_dimension_naxis2,obsID,planeID,productID,dataProductType,quality_flag,position_bounds_points,proposal_keywords,collection,type,target_name,telescope_name,time_bounds_lower
,deg,deg,arcsec,,,,,,,,deg,,,,,,d
int32,object,float64,float64,int64,int64,bytes36,bytes36,bytes64,bytes128,bytes16,object,object,bytes32,bytes32,bytes32,bytes32,float64
2,[17.089044308686464 6.179624184699989 16.102294508268866 6.175765189727662 16.097523694475832 7.1662126451615995 16.207496294384118 7.1726066347764235 17.088469488471336 7.17754910258159],1.4010581571637863,0.186124380919742,18960,19288,00000000-0000-0000-d4a7-013ed27e3900,00000000-0000-0000-16dd-013ed281f9e0,869870p,image,,[17.089044308686457 6.179624184699989 16.102294508268844 6.175765189727656 16.097523694475836 7.166212645161593 16.207496294384143 7.172606634776415 17.088469488471343 7.177549102581608],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.3964907
2,[17.08680792324209 7.169327594766717 17.088182765663575 6.178778578955335 16.103033843988143 6.175641983503534 16.097523694475832 7.1662126451615995 16.97658373792005 7.174342410670307],1.3939799245011717,0.18603218540449784,18944,19226,00000000-0000-0000-fdaf-013ed27e3e62,00000000-0000-0000-af94-013ed2827ded,869882p,image,,[17.086807923242077 7.169327594766699 17.088182765663568 6.178778578955345 16.10303384398813 6.175641983503539 16.09752369447582 7.166212645161593 16.976583737920052 7.174342410670292],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.4083618
2,[17.086670749275502 6.1788494744227425 16.10192410471798 6.168266882869949 16.097523694475832 7.1662126451615995 17.084542070816205 7.173698078762894],1.4012132393008745,0.18603657249693065,18915,19358,00000000-0000-0000-9729-013ed27eea40,00000000-0000-0000-7761-013ed28294dd,869894p,image,,[17.086670749275477 6.178849474422719 16.10192410471799 6.168266882869958 16.097523694475836 7.166212645161593 17.084542070816227 7.173698078762911],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.4196013
2,[17.08644182490072 6.178730356804927 16.10192410471798 6.168266882869949 16.097069533628247 7.169911415565479 17.084946054769546 7.173407989537367],1.4012848509702989,0.18600928684540355,18919,19353,00000000-0000-0000-6ac3-013ed27ff52e,00000000-0000-0000-28e2-013ed282e247,869906p,image,,[17.086441824900714 6.178730356804934 16.101924104717966 6.168266882869958 16.097069533628243 7.169911415565494 17.084946054769542 7.173407989537381],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.4311179
2,[17.09023544093668 6.1716700786633565 16.101375217012983 6.175583331346593 16.096894126818416 7.170282922285291 17.08710654735683 7.176290862980157],1.4037939181544752,0.18606638378128082,18995,19342,00000000-0000-0000-179f-013ed2828efe,00000000-0000-0000-84cb-013ed285d2a1,869918p,image,,[17.090235440936684 6.171670078663368 16.101375217012972 6.175583331346601 16.096894126818402 7.17028292228531 17.087106547356836 7.176290862980167],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.4426172
2,[17.086186307313085 6.177582784339037 16.980911410111567 6.170591584795881 16.101391112741805 6.174954721637882 16.096569905653407 7.169101927727475 17.086637227332847 7.175617083440133],1.3996131808634202,0.18597880826348204,18923,19353,00000000-0000-0000-3d38-013ed2836b5d,00000000-0000-0000-40bf-013efe50b43d,869930p,image,,[17.08618630731311 6.177582784339009 16.98091141011156 6.170591584795872 16.101391112741812 6.174954721637903 16.096569905653382 7.169101927727482 17.08663722733285 7.1756170834401445],,CFHT,OBJECT,Main12,CFHT 3.6m,53992.4544582
2,[17.08564893322855 6.178501927180807 16.10192410471798 6.168266882869949 16.096160815956672 7.17028370997764 17.083661316252822 7.173467413024072],1.400438660041335,0.186125836699184,18921,19353,00000000-0000-0000-29f7-013ed28525d7,00000000-0000-0000-0ef0-013ed288a5b1,869942p,image,,[17.08564893322858 6.178501927180804 16.101924104717988 6.168266882869943 16.096160815956672 7.170283709977625 17.083661316252826 7.173

We are interested in data from both the SSOIS table (i.e. the Object_RA and Object_Dec) and the CADC table (i.e. the time_bounds_lower and the ability to use the `get_data_urls` function). To keep all the data together, we can merge the two tables, since the SSOIS `Image` column and the CADC `productID` column map to each other.

In [8]:
from astropy.table import Table, join

table.rename_column('Image', 'productID')
joined_table = join(table, results, keys='productID')
joined_table['url'] = cadc.get_data_urls(joined_table)

joined_table['url'][0:5]

https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869870p.fits.fz?RUNID=s9aipcteda96oo7d
https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869882p.fits.fz?RUNID=zj55bpvaxkiihub6
https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869894p.fits.fz?RUNID=k07w0u9kkdr8o8ia
https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869906p.fits.fz?RUNID=pjsbl2ol2b8itc30
https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/869918p.fits.fz?RUNID=byqwe6ccbdlk35yc


In [9]:
from astropy.coordinates import SkyCoord

def get_selection(table, date_range):
    return table[(results['time_bounds_lower'] > date_range[0]) & 
                        (results['time_bounds_lower'] < date_range[1])]

def get_cutout_params(table):
    ra_max, ra_min = table['Object_RA'].max(), table['Object_RA'].min()
    dec_max, dec_min = table['Object_Dec'].max(), table['Object_Dec'].min()

    ra_centre = (ra_max + ra_min)/2 
    dec_centre = (dec_max + dec_min)/2

    ra_radius =  abs(ra_max - ra_min)/2 
    dec_radius =  abs(dec_max - dec_min)/2

    radius = round(max(ra_radius, dec_radius), 2)
    
    return ra_centre, dec_centre, radius


In [10]:
import re
from astropy.wcs import WCS
from astropy.io import fits
import urllib

def get_cutout_urls(url, ra, dec, radius=0.02, verbose=False):
    uri = 'ad:' + \
        re.findall(
            "https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/(.+)\?RUNID=", url)[0]

    cutout_region_string = 'Circle ICRS {ra} {dec} {radius}'.format(
        ra=ra, dec=dec, radius=radius)
    params_dict = {'uri': uri, 'cutout': cutout_region_string}

    base_url = 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/caom2ops/cutout'

    url_params = urllib.parse.urlencode(params_dict)
    data_url = '{}?{}'.format(base_url, url_params)
    if verbose:
        print(data_url)
    return data_url


# Using photutils

In [117]:
%matplotlib widget
import matplotlib.pyplot as plt
import warnings
from matplotlib.patches import Circle

from photutils import CircularAperture, aperture_photometry
import matplotlib.gridspec as gridspec
from astropy.visualization import LinearStretch, ImageNormalize, ZScaleInterval

#TODO: Figure out what theses do and where to use them
# plt.ioff()
# plt.clf()

# Supress fits processing warnings
warnings.simplefilter('ignore')

# Aperture results widget
def make_aperture_text_widget():
    aperture_widget = widgets.HTML(
        placeholder='Click and drag on a subplot to select an area to analyze',
        description='Aperture Data:',
        disabled=False,
        style={'description_width': 'initial'}
    )
    aperture_widget.layout.width = '800px'
    return aperture_widget

aperture_widget = make_aperture_text_widget()


# Range slider widget
def make_date_range_slider(results):
    start_date = math.floor(results['time_bounds_lower'].min())
    end_date = math.ceil(results['time_bounds_lower'].max())

    dates = Time(np.arange(start_date, end_date+1, 1), format='mjd', out_subfmt='date')
    options = [(' {} '.format(date.iso), date.mjd) for date in dates]
    range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=(0, 1), #TODO: Should check that this is not more than length of list
        description='Date Range:',
        disabled=False,
        continuous_update=False,
    )
    range_slider.layout.width = '800px'
    return range_slider

def update_data(change):
    #TODO: Check for overlap between old and new rather than re-grabbing everything
    print(change)
#     data_list = get_data_list(joined_table, change.new)
#     if data_list:
#         fig = kbo_figure(data_list) 
#         fig.canvas.draw()
#         fig.canvas.flush_events()

range_slider = make_date_range_slider(results)
range_slider.observe(update_data, names='value')


def get_data_list(joined_table, range_value, ext=2):
    #TODO: Choose larger extension rather than just ext=2 lol
    data_list = []
    selection = get_selection(joined_table, range_value)
    if len(selection) < 1:
        print("Range is too small")
        return None
    ra, dec, radius = get_cutout_params(selection)
    selection['cutout_url'] = [get_cutout_urls(url, ra, dec, radius=radius*2) for url in selection['url']]

    for url, ra, dec, date in zip(selection['cutout_url'], selection['Object_RA'], 
                                  selection['Object_Dec'], selection['time_bounds_lower']):
        with fits.open(url, ignore_missing_end=True) as hdulist:
            data = hdulist[ext].data
            wcs = WCS(hdulist[ext].header)
            date_fmtd = Time(date, format='mjd', out_subfmt='date_hm').iso
            data_list.append({'image_data': data, 'wcs': wcs, 'ra': ra, 'dec': dec, 'date': date_fmtd})
            
    return data_list
    

#TODO: TRY with aplpy
def kbo_figure(data_list, kbo_name):
    #https://matplotlib.org/3.1.0/gallery/subplots_axes_and_figures/axes_zoom_effect.html
    #sphx-glr-gallery-subplots-axes-and-figures-axes-zoom-effect-py
    # TODO: Add circles around KBO
    
    p_center = None
    global data_dict
    
    data_dict = {}
    
    def onclick(event):
        # Check if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != '' or event.inaxes is None): return

        global p_center

        #     if circle and circle.contains(event):
        #         circle.set_color('orange')
        p_center = (event.xdata, event.ydata)

    def onrelease(event):
        # Check if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != '' or event.inaxes is None): return
    
        global p_center
        global kbo_fig
        global data_dict
        
        p_outside = (event.xdata, event.ydata)
        radius = math.sqrt((p_center[0] - p_outside[0])**2 +
                           (p_center[1] - p_outside[1])**2)
        
        if radius == 0.0: return
        
#         # Remove old circles
        for ax in kbo_fig.axes:
            for artist in ax.artists:
                if isinstance(artist, plt.Circle):
                    artist.remove()

        # Draw new circles
        circle = plt.Circle(p_center, radius, color='black', fill=False)
        ax = event.inaxes
        ax.add_artist(circle)
        kbo_fig.canvas.draw()

        aperture = CircularAperture(p_center, r=radius)
        image_data = data_dict[ax.colNum]
        phot_table = aperture_photometry(image_data, aperture)
        aperture_widget.value = ' '.join(phot_table['aperture_sum'].pformat(html=True))

    def plot_data(kbo_fig, data_list):
        n = len(data_list)
        global data_dict
        cid_press = kbo_fig.canvas.mpl_connect('button_press_event', onclick)
        cid_release = kbo_fig.canvas.mpl_connect('button_release_event', onrelease)
        wcs_trans = data_list[0]['wcs']
        gs = gridspec.GridSpec(1, n, figure=kbo_fig, wspace=0.1)

        for idx, data in enumerate(data_list):
            try:

                # Normalize and plot the data
                ax = plt.subplot(gs[idx], 
                                 projection=wcs_trans, adjustable='box', aspect='equal')
                data_dict[idx] = data['image_data']

                image_data_norm = ImageNormalize(data['image_data'],
                                                 interval=ZScaleInterval(),
                                                 stretch=LinearStretch())
                ax.imshow(data['image_data'],
                          norm=image_data_norm,
                          transform=ax.get_transform(data['wcs']),
                          cmap='gray')
                
                ax.set_title(data['date'], fontsize=10)
                
                # Add yellow circle around KBO position
                c = Circle((data['ra'], data['dec']), 0.004, edgecolor='yellow', facecolor='none',
                      transform=ax.get_transform('world'))
                ax.add_patch(c)

                # Add grid and remove axis labels
                ax.coords.grid(color='white', ls='solid')
                ra, dec = ax.coords['ra'], ax.coords['dec']
                ra.set_ticklabel_visible(False)
                dec.set_ticklabel_visible(False)
                ra.set_axislabel('')
                dec.set_axislabel('')

            except ValueError as ex:
                print('Value Error: %s' % ex)
                continue

        axes = kbo_fig.get_axes()
        axes[0].get_shared_x_axes().join(*axes)
        axes[0].get_shared_y_axes().join(*axes)
        plt.subplots_adjust(left=0.05, right=0.95, top=0.9)
        plt.tight_layout(pad=0.5)

    kbo_fig = plt.figure(figsize=(9, 5))
    kbo_fig.suptitle('Images of KBO {}'.format(kbo_name), fontsize=14, fontweight='bold', y=0.94)
    plot_data(kbo_fig, data_list)
    return kbo_fig



In [115]:
from photutils import CircularAperture
positions = [(30., 30.), (40., 40.)]
apertures = CircularAperture(positions, r=3.)

import numpy as np
from photutils import aperture_photometry
data = np.ones((100, 100))
phot_table = aperture_photometry(data, apertures)
phot_table['aperture_sum'].info.format = '%.6g'  # for consistent table output
print(phot_table['aperture_sum']])


 id aperture_sum
--- ------------
  1      28.2743
  2      28.2743


In [116]:

aperture_widget.value = ' '.join(phot_table[['id', 'aperture_sum']].pformat(html=True))
aperture_widget

HTML(value='<table id="table139913696577632"> <thead><tr><th>id</th><th>aperture_sum</th></tr></thead> <tr><td…

In [50]:
range_slider

SelectionRangeSlider(continuous_update=False, description='Date Range:', index=(0, 1), layout=Layout(width='80…

In [13]:
data_list = get_data_list(joined_table, range_slider.value)


print(len(joined_table))
print(len(data_list))

95
17


In [118]:
# plt.ioff()
kbo_fig = kbo_figure(data_list[::3], kbo_name)

FigureCanvasNbAgg()

In [119]:
# Display Block
# Organize the widgets presentably

#kbo_figure()
#plt.clf()
plt.ion()
b_layout = widgets.Layout(align_items='center', align_content = 'center', border='none', justify_content = 'center', 
                          width = '100%')

big_box = widgets.VBox([range_slider, kbo_fig.canvas, aperture_widget], layout = b_layout)

display(big_box)


In [ ]:
#X  TODO: Get interactive toolbar working
#X TODO: Better layout (not sp tall and skinny and spreadout)
# TODO: Get date range slider to replot current fig (not make new figs!)
# TODO: Make aperture data more visually nice